In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from collections import Counter
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("GoogleSheetsToDelta").getOrCreate()

# Service account credentials (Replace with actual values)
service_account_info = {
    "type": "service_account",
    "project_id": "YOUR_PROJECT_ID",
    "private_key_id": "YOUR_PRIVATE_KEY_ID",
    "private_key": "YOUR_PRIVATE_KEY",
    "client_email": "YOUR_CLIENT_EMAIL",
    "client_id": "YOUR_CLIENT_ID",
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": "YOUR_CLIENT_CERT_URL",
    "universe_domain": "googleapis.com"
}

# Google Sheets API authentication
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_dict(service_account_info, scope)
client = gspread.authorize(credentials)

# Google Sheets data mapping
sheets_data = [
    {
        "sheet_url": "YOUR_GOOGLE_SHEET_URL_1",
        "worksheet_name": "YOUR_WORKSHEET_NAME_1",
        "table_name": "YOUR_DELTA_TABLE_NAME_1"
    },
    {
        "sheet_url": "YOUR_GOOGLE_SHEET_URL_2",
        "worksheet_name": "YOUR_WORKSHEET_NAME_2",
        "table_name": "YOUR_DELTA_TABLE_NAME_2"
    },
    {
        "sheet_url": "YOUR_GOOGLE_SHEET_URL_3",
        "worksheet_name": "YOUR_WORKSHEET_NAME_3",
        "table_name": "YOUR_DELTA_TABLE_NAME_3"
    }
    # Add more sheets here if needed
]

In [ ]:
# Function to rename duplicate column names
def rename_columns(columns):
    counter = Counter()
    new_columns = []
    
    for col in columns:
        col = col.strip() or "unnamed"  # If empty, rename to 'unnamed'

        if counter[col]:  # If duplicate, append .1, .2, etc.
            new_col = f"{col}.{counter[col]}"
        else:
            new_col = col
        
        new_columns.append(new_col)
        counter[col] += 1  # Increase count for next duplicate
        
    return new_columns

In [ ]:

for item in sheets_data:
    spreadsheet = client.open_by_url(item["sheet_url"])
    worksheet = spreadsheet.worksheet(item["worksheet_name"])

    data = worksheet.get_all_values()

    pandas_df = pd.DataFrame(data)
    pandas_df.columns = rename_columns(pandas_df.iloc[0])

    # Drop the first row (since it's now the header)
    pandas_df = pandas_df[1:].reset_index(drop=True)
    pandas_df = pandas_df.astype(str)

    spark_df = spark.createDataFrame(pandas_df)
    
    # Drop existing table if it exists
    spark.sql(f"DROP TABLE IF EXISTS {item['table_name']}")

    # Save as Delta Table in Databricks
    spark_df.write.format("delta").option("delta.columnMapping.mode", "name").mode("overwrite").saveAsTable(item['table_name'])